In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

  # Install the latest version of torchtext library quietly without showing output
!pip install torchtext -qq
!pip install transformers evaluate wandb datasets accelerate -U -qq ## NEW LINES ##
basepath = '/content/drive/MyDrive/NLP/'

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.4 MB/s eta 0:00:00


In [2]:
from pathlib import Path
# wandb library
import wandb
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm
from sklearn import metrics
import torch
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)
import os, sys
os.environ['TOKENIZERS_PARALLELISM']='false'

from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

import transformers
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, DistilBertForSequenceClassification
from transformers import TrainingArguments, Trainer, set_seed

import datasets
from datasets import Dataset, DatasetDict

import evaluate

import torch

!git lfs install

Git LFS initialized.


In [3]:
base_folder = Path(basepath)
data_folder = base_folder/'Homework/Homework7'
model_folder = base_folder/'models/nlp_spring_2024/ed/nn'
custom_functions = base_folder/'custom-functions'

In [4]:
model_folder.mkdir(exist_ok=True, parents = True)

In [5]:
sys.path.append('/content/drive/MyDrive/NLP/custom-functions')
import CustomPreprocessorSpacy as cp

In [6]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [7]:
train_data = pd.read_csv(data_folder/ 'train.csv')
test = pd.read_csv(data_folder/ 'test.csv')

In [8]:
train_data.drop(columns=["ID"],inplace=True)
y_test = test.drop(['ID','Tweet'],axis=1)
for column in y_test:
    y_test[column] = y_test[column].apply(lambda x: 0 if x == "NONE" else 1)
y_test = y_test.apply(lambda x: list(map(int, x)))
test_data = pd.concat([test['Tweet'],y_test],axis=1)

train_data.head()


,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0


In [9]:
test_data.head()

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,@Adnan__786__ @AsYouNotWish Dont worry Indian ...,0,0,0,0,0,0,0,0,0,0,0
1,"Academy of Sciences, eschews the normally sobe...",0,0,0,0,0,0,0,0,0,0,0
2,I blew that opportunity -__- #mad,0,0,0,0,0,0,0,0,0,0,0
3,This time in 2 weeks I will be 30... 😥,0,0,0,0,0,0,0,0,0,0,0
4,#Deppression is real. Partners w/ #depressed p...,0,0,0,0,0,0,0,0,0,0,0


In [10]:
labels = [x for x in train_data.columns if x not in ['Tweet']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
NUM_OF_LABELS = len(labels)

In [11]:
NUM_OF_LABELS

11

In [12]:
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
train_testEval = train_dataset.train_test_split(train_size=0.80)
val_eval = train_testEval['test']

ds = DatasetDict({
    'train' : train_testEval['train'],
    'val' : val_eval,
    'test':test_dataset

})

print("Training Dataset Shape:", ds['train'].shape)
print("Evaluation Dataset Shape:", ds['val'].shape)
print("Testing Dataset Shape",ds['test'].shape)
ds['train'][25]

Training Dataset Shape: (6179, 12)
Evaluation Dataset Shape: (1545, 12)
Testing Dataset Shape (3259, 12)


{'Tweet': "Yo Yo Yo,my name is #DarthVader \\nI feel like I need to puff on my inhaler (I'm no rapper but that was some sick bars)  #bars #rap",
 'anger': 0,
 'anticipation': 0,
 'disgust': 1,
 'fear': 0,
 'joy': 0,
 'love': 0,
 'optimism': 0,
 'pessimism': 0,
 'sadness': 1,
 'surprise': 0,
 'trust': 0}

In [13]:
set_seed = 42
NUM_OF_EPOCHS = 10

BATCH_SIZE = 64
LEARNING_RATE = 1e-4

MAX_LENGTH = 128
MODEL_CKPT = "distilbert-base-uncased"
MODEL_NAME = MODEL_CKPT + "-reviews_multilabel_clf_v2"

METRIC_NAME = "f1_macro"
DEVICE = torch.device("cpu")
print(f"This project will utilize the {DEVICE} to train and evaluate.")
print(f"This project will have a maximum length for the input text of {MAX_LENGTH} tokens.")

This project will utilize the cpu to train and evaluate.
This project will have a maximum length for the input text of 128 tokens.


In [14]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT)

def data_preprocessor(examples):
	# take a batch of text
	text = examples["Tweet"]
	# encode them
	encoded_data = tokenizer(text, padding="max_length", truncation=True, max_length=MAX_LENGTH)
	# add labels
	labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
	# create numpy array of shape (batch_size, num_labels)
	labels_matrix = np.zeros((len(text), len(labels)))
	# fill numpy array
	for idx, label in enumerate(labels):
		labels_matrix[:, idx] = labels_batch[label]

	encoded_data["labels"] = labels_matrix.tolist()

	return encoded_data

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [15]:
encoded_ds = ds.map(data_preprocessor, batched=True, load_from_cache_file=False)
encoded_ds.set_format("torch")

Map:   0%|          | 0/6179 [00:00<?, ? examples/s]

Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [16]:
print(f"The shape of the training dataset is: {encoded_ds['train'].shape}")

print(f"The shape of the evaluation dataset is: {encoded_ds['val'].shape}")
print(f"The shape of the test dataset is: {encoded_ds['test'].shape}")

The shape of the training dataset is: (6179, 15)
The shape of the evaluation dataset is: (1545, 15)
The shape of the test dataset is: (3259, 15)


In [17]:
model = (DistilBertForSequenceClassification.from_pretrained(MODEL_CKPT,
                                                             num_labels=NUM_OF_LABELS,
                                                             problem_type="multi_label_classification",
                                                             id2label=id2label,
                                                             label2id=label2id)
         ).to(DEVICE)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
def multi_label_metrics(predictions,
                        labels,
                        threshold=0.5):
    '''
    This function calculates & returns metrics
    for a multilabel classification analysis.
    '''

    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro_average = f1_score(y_true=y_true,y_pred=y_pred,average ='macro')

    #roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return metrics as dictionary
    metrics = {'f1_micro': f1_micro_average,
               'f1_macro':f1_macro_average,
               #'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    preds = preds[0] if isinstance(preds,
                    tuple) else preds
    results = multi_label_metrics(
                    predictions=preds,
                    labels=labels)
    return results

In [19]:
args = TrainingArguments(
    output_dir=model_folder,
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=50,  # Perform evaluation every 10 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=50,
    save_total_limit=1,# Save a model checkpoint every 10 training steps
    learning_rate=LEARNING_RATE,
    lr_scheduler_type='linear',
    num_train_epochs=NUM_OF_EPOCHS,
    optim='adamw_torch',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    load_best_model_at_end=True,
    metric_for_best_model=METRIC_NAME,
    warmup_steps = 0,
    greater_is_better=True,
    weight_decay=0.01,
    disable_tqdm=False,
    logging_first_step=True,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name='Tweet_Emotion_DISTILLBERT',
)

In [20]:
trainer = Trainer(
    model,
    args,

    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [21]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [22]:
trainer.train()

wandb: Currently logged in as: rangareddy-bhargav. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,F1 Micro,F1 Macro,Accuracy
50,0.690400,0.390343,0.514936,0.244744,0.175405
100,0.690400,0.337866,0.625605,0.413177,0.242718
150,0.690400,0.329041,0.642626,0.446261,0.239482
200,0.690400,0.315598,0.659196,0.489595,0.258252
250,0.690400,0.323355,0.662012,0.497313,0.253074
300,0.690400,0.316783,0.662733,0.503276,0.260194
350,0.690400,0.334457,0.668275,0.524667,0.258900
400,0.690400,0.336059,0.665740,0.510043,0.256958
450,0.690400,0.346668,0.665014,0.549949,0.238835
500,0.263700,0.359503,0.666369,0.529672,0.239482


TrainOutput(global_step=970, training_loss=0.18184316410231835, metrics={'train_runtime': 840.4258, 'train_samples_per_second': 73.522, 'train_steps_per_second': 1.154, 'total_flos': 2046618574855680.0, 'train_loss': 0.18184316410231835, 'epoch': 10.0})

In [23]:
trainer.evaluate()

{'eval_loss': 0.3466684818267822,
 'eval_f1_micro': 0.6650138544553013,
 'eval_f1_macro': 0.5499492289906406,
 'eval_accuracy': 0.23883495145631067,
 'eval_runtime': 6.06,
 'eval_samples_per_second': 254.949,
 'eval_steps_per_second': 4.125,
 'epoch': 10.0}

In [24]:
valid_output = trainer.predict(encoded_ds["val"])

In [25]:
valid_output._fields

('predictions', 'label_ids', 'metrics')

In [26]:
valid_preds = np.argmax(valid_output.predictions, axis=-1)
valid_labels = np.array(valid_output.label_ids)


In [27]:
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 450.


In [28]:
model_folder

PosixPath('/content/drive/MyDrive/NLP/models/nlp_spring_2024/ed/nn')

In [29]:
model_checkpoint = str(model_folder) + '/checkpoint-' + best_model_checkpoint_step

In [30]:
valid_results = trainer.evaluate(encoded_ds["val"])

In [31]:
valid_results

{'eval_loss': 0.3466684818267822,
 'eval_f1_micro': 0.6650138544553013,
 'eval_f1_macro': 0.5499492289906406,
 'eval_accuracy': 0.23883495145631067,
 'eval_runtime': 6.0031,
 'eval_samples_per_second': 257.367,
 'eval_steps_per_second': 4.165,
 'epoch': 10.0}

In [32]:
model = DistilBertForSequenceClassification.from_pretrained(model_checkpoint)
model.to('cuda')
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [33]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to=[]
)

In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=encoded_ds['test'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [35]:
encoded_ds['test']['labels']

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [36]:
test_results = trainer.evaluate()

In [37]:
test_results = trainer.predict(encoded_ds['test'])

# The predictions object contains various fields including predictions
logits = test_results.predictions

# Apply sigmoid to get the probabilities
probs = torch.sigmoid(torch.Tensor(logits))

# Apply a threshold to get the binary predictions
threshold = 0.5
y_pred = (probs >= threshold).int().numpy()

In [38]:
y_pred_df = pd.DataFrame(y_pred,columns=labels)
output_submission = pd.concat([test['ID'],y_pred_df],axis=  1)
output_submission

,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,1,0,1,1,0,0,0,0,0,0,0
1,2018-03739,1,0,1,0,0,0,0,0,1,0,0
2,2018-00385,1,0,1,0,0,0,0,0,1,0,0
3,2018-03001,0,0,0,0,1,0,1,0,0,0,0
4,2018-01988,0,0,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3254,2018-03848,1,0,1,0,0,0,0,0,0,0,0
3255,2018-00416,0,1,0,1,0,0,0,0,0,0,0
3256,2018-03717,1,0,1,0,0,0,0,0,0,0,0
3257,2018-03504,0,0,0,0,1,1,1,0,0,0,0


In [39]:
output_submission.to_csv('/content/drive/MyDrive/NLP/output_submission.csv', index=False)

In [40]:
wandb.finish()

eval/accuracy,▁▇▆█▇███▆▆▅▆▅▄▅▅▅▅▅▆▆
eval/f1_macro,▁▅▆▇▇▇▇▇███████▇█████
eval/f1_micro,▁▆▇████████████▇██▇██
eval/loss,▅▂▂▁▁▁▂▂▃▃▄▅▆▆▇▇███▃▃
eval/runtime,▁█▆▅▇▆█▆▇▆▆▆▇▇▇▆▆▆█▆▅
eval/samples_per_second,█▁▃▄▂▂▁▃▂▃▃▃▂▂▂▃▂▃▁▃▄
eval/steps_per_second,█▁▃▄▂▂▁▃▂▃▃▃▂▂▂▃▂▃▁▃▄
train/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇████
train/global_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇████
train/grad_norm,▁█
train/learning_rate,█▁
